# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [57]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [58]:
weather_path = "../WeatherPy/output_data/cities.csv"
weather_df = pd.read_csv(weather_path)

weather_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ibipora,-23.27,-51.05,75.20,78,0,2.24,BR,1603585829
1,chokurdakh,70.63,147.92,14.49,97,95,7.25,RU,1603585683
2,carnarvon,-24.87,113.63,73.40,88,90,16.11,AU,1603585673
3,new norfolk,-42.78,147.06,54.00,48,79,1.99,AU,1603585816
4,punta arenas,-53.15,-70.92,42.80,93,100,14.99,CL,1603585650
...,...,...,...,...,...,...,...,...,...
558,eregli,37.51,34.05,56.95,36,87,2.04,TR,1603585986
559,battle creek,42.32,-85.18,41.00,65,75,4.70,US,1603585800
560,vysokopillya,47.49,33.53,51.80,76,0,4.85,UA,1603585986
561,patacamaya,-17.23,-67.92,53.17,59,5,6.29,BO,1603585987


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [59]:
gmaps.configure(api_key=g_key)

In [60]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lon"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
humidity = weather_df["Humidity"].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 max_intensity=weather_df["Humidity"].max())

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
hotel_df = weather_df[weather_df['Max Temp'] >= 74] 
hotel_df = hotel_df[hotel_df['Max Temp'] <= 80] 
hotel_df = hotel_df[hotel_df['Wind Speed'] < 10]
hotel_df = hotel_df[hotel_df['Cloudiness'] == 0]
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,ibipora,-23.27,-51.05,75.20,78,0,2.24,BR,1603585829,
155,bilma,18.69,12.92,74.05,18,0,9.40,NE,1603585875,
182,arlit,18.74,7.39,77.74,16,0,8.01,NE,1603585682,
223,harper,4.38,-7.72,77.41,88,0,6.06,LR,1603585895,
272,tessalit,20.20,1.01,78.03,17,0,7.52,ML,1603585850,
338,sedhiou,12.71,-15.56,77.29,79,0,4.34,SN,1603585929,
348,san juan,-31.54,-68.54,76.68,46,0,3.33,AR,1603585932,
402,surat,21.17,72.83,75.94,50,0,5.44,IN,1603585813,
441,kolondieba,11.09,-6.89,74.71,48,0,2.44,ML,1603585955,
498,sur,22.57,59.53,74.43,54,0,8.99,OM,1603585816,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [48]:
hotel_df['Hotel Name'] = ""
for index, row in hotel_df.iterrows():
    # geocoordinates
    target_coordinates = f'{row["Lat"]},{row["Lon"]}'
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "rankby": "distance",
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    call = requests.get(base_url, params=params)
    if call:
        response = call.json()['results']
        if(len(response) > 1):
            hotel_df.loc[index,'Hotel Name'] = response[0]['name']

In [55]:
heat_layer.max_intensity = hotel_df['Humidity'].max()


In [51]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [52]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))